In [1]:
import pandas as pd
import numpy as np

In [4]:
# title budget genres country main_charactor director vote_average vote_count
df = pd.read_csv('data/movie_data_final.csv')

In [5]:
# 로지스틱 회귀 모델(sigmoid) : weight초기화, dropout, ReLU, batch nomalization
# 결과 : 흥행성공, 흥행 실패, 전체영화 의 평균 정확도
# 1. 데이터셋 준비하기
ID = []
for idx in range(5302):
    ID.append([title_ID[idx], *budget[idx], genre_ID[idx], country_ID[idx], char_ID[idx], dir_ID[idx], result[idx]])
X = np.array(ID)

,title,budget,genres,country,main_charactor,director,vote_average,vote_count
0,{616037: 'Thor: Love and Thunder'},250000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",{3894: 'Christian Bale'},{55934: 'Taika Waititi'},6.772,1752
1,{438148: 'Minions: The Rise of Gru'},85000000,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...","[{'iso_3166_1': 'US', 'name': 'United States o...",{124747: 'Pierre Coffin'},{8023: 'Kyle Balda'},7.795,1166
2,{507086: 'Jurassic World Dominion'},165000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...","[{'iso_3166_1': 'US', 'name': 'United States o...",{18997: 'Bryce Dallas Howard'},{930707: 'Colin Trevorrow'},7.117,2622
3,{361743: 'Top Gun: Maverick'},170000000,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",{996701: 'Miles Teller'},{86270: 'Joseph Kosinski'},8.334,1852
4,{756999: 'The Black Phone'},18800000,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",{1530558: 'Madeleine McGraw'},{55499: 'Scott Derrickson'},7.975,1671


In [ ]:
# 장르 id와 국가별 id 길이 맞추기
# ex) [1, 2, 3], [1, 4, x] -> [1, 2, 3], [1, 4, 0]
# 길이의 최대값 구하기 7개
mx = 0
for length in genre_ID:
    if mx < len(length):
        mx = len(length)


# 최대가 아닌 길이들 combination
# ex) [1, 0, 0] => [1, 0, 0], [0, 1, 0], [0, 0, 1]

In [ ]:
# y_train : 흥행했는지(1) 안했는지(0)
from sklearn.model_selection import train_test_split
y = X[:, 6:]
X = X[:, :-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


In [ ]:
X_train[0]


In [ ]:
# 2. 정규화 과정
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
